<a href="https://colab.research.google.com/github/GinkGoPi/researches/blob/main/token-stake/booster_logic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Staking with lock to booster

通过质押token并且锁仓一定时间，可以获取boost的收益

奖励部分的逻辑与`SimepleStakingRewards`一致，只是其中用户质押的数据需要按照`BoostPool`中的**份额**来替代计算

核心的问题：
- 质押的数量转换成份额数
- **Lock**周期带来的Boost收益系数


In [ ]:
import calendar
import time
import datetime
from datetime import datetime
from datetime import timedelta

In [ ]:
calendar.timegm(time.gmtime())

1675305783

In [ ]:
class StakingRewards(object):
    
    def __init__(self) -> None:
        self.stakingToken = 'stakingToken'
        self.rewardsToken = 'rewardsToken'

        self.periodFinish = 0
        self.lastUpdateTime = 0
        self.duration = 1 * 60*60*24 # seconds

        self.rewardRate = 0
        self.rewardPerTokenStored = 0

        self.userRewardPerTokenPaid = {'boostPool': 0, 'alice': 0, 'bob': 0, 'kite': 0}
        self.rewards = {'boostPool': 0, 'alice': 0, 'bob': 0, 'kite': 0}

        # Note: this data will be from `BoostPool`
        # self.totalSupply = 0
        # self.balanceOf = {'boostPool': 0, 'alice': 0, 'bob': 0, 'kite': 0}
    
    # def stake(self, address: str, amount: int):
    #     self.updateReward(address)
    #     self.balanceOf[address] += amount
    #     self.totalSupply += amount
    
    # def withdraw(self, to: str, amount: int):
    #     self.updateReward(to)
    #     self.balanceOf[to] -= amount
    #     self.totalSupply -= amount
    #     return amount
      
    def updateReward(self, _account: str):
        self.rewardPerTokenStored = self.rewardPerToken()
        self.lastUpdateTime = self.lastTimeRewardApplicable()

        if _account != "address_0":
            self.rewards[_account] = self.earned(_account)
            self.userRewardPerTokenPaid[_account] = self.rewardPerTokenStored
    
    def rewardPerToken(self):
        if self.totalSupply == 0:
            return self.rewardPerTokenStored
        else:
            return self.rewardPerTokenStored + (self.rewardRate * (self.lastTimeRewardApplicable() - self.lastUpdateTime) * 1e18) / self.totalSupply

    def earned(self, _account: str):
        return ((self.balanceOf[_account] *
                (self.rewardPerToken() - self.userRewardPerTokenPaid[_account])) / 1e18) + self.rewards[_account]
    
    def update(self, account):
        # !! must in boostpool, to update account reward 
        self.updateReward(account)

    def getReward(self, to: str):
        """Gain reward token"""
        self.updateReward(to)
        reward = self.rewards[to]
        if reward > 0:
            self.rewards[to] = 0
            # rewardsToken.transfer(to, reward)
        
        return reward

    def notifyRewardAmount(self, _amount: int):
        self.updateReward("address_0")
        if self.blockTimestamp() >= self.periodFinish:
            self.rewardRate = _amount / self.duration
        else:
            remainingRewards = (self.periodFinish - self.blockTimestamp()) * self.rewardRate
            self.rewardRate = (_amount + remainingRewards) / self.duration
        

        assert(self.rewardRate > 0, "reward rate = 0")

        self.lastUpdateTime = self.blockTimestamp()
        self.periodFinish = self.blockTimestamp() + self.duration
        
    
    def lastTimeRewardApplicable(self):
        return self.periodFinish if self.periodFinish <= self.blockTimestamp() else self.blockTimestamp()

    def blockTimestamp(self):
        """Akin to `block.timestamp`"""
        return calendar.timegm(time.gmtime())

<>:69: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:69: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-49-c96011e66ff7>:69: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(self.rewardRate > 0, "reward rate = 0")


## BoostPool logic

In [13]:
class UserInfo(object):
    def __init__(self, 
        shares=0, 
        lastDepositedTime=0, 
        tokenAtLastUserAction=0, 
        lastUserActionTime=0,
        lockStartTime=0,
        lockEndTime=0,
        userBoostedShare=0,
        locked=False,
        lockedAmount=0):
      self.shares = shares
      self.lastDepositedTime = lastDepositedTime
      self.tokenAtLastUserAction = tokenAtLastUserAction
      self.lastUserActionTime = lastUserActionTime
      self.lockStartTime = lockStartTime
      self.lockEndTime = lockEndTime
      self.userBoostedShare = 0
      self.locked = locked
      self.lockedAmount = lockedAmount
    
    def getAll(self):
      return {
          "shares": format(self.shares, '.2f'),
          "lastDepositedTime": self.lastDepositedTime,
          "tokenAtLastUserAction": self.tokenAtLastUserAction,
          "lastUserActionTime": self.lastUserActionTime,
          "lockStartTime": self.lockStartTime,
          "lockEndTime": self.lockEndTime,
          "userBoostedShare": format(self.userBoostedShare, '.2f'),
          "locked": self.locked,
          "lockedAmount": format(self.lockedAmount, '.2f')
      }


MAX_PERFORMANCE_FEE = 2000 # 20%
MAX_WITHDRAW_FEE = 500 # 5%
MAX_OVERDUE_FEE = 100 * 1e10 # 100%
MAX_WITHDRAW_FEE_PERIOD = 7 *60*60*24 # 1 week
MIN_LOCK_DURATION = 7 *60*60*24
MAX_LOCK_DURATION_LIMIT = 1000 * 60*60*24
BOOST_WEIGHT_LIMIT = 5000 * 1e10
PRECISION_FACTOR = 1e12
PRECISION_FACTOR_SHARE = 1e28;
MIN_DEPOSIT_AMOUNT = 0.00001 *1e18
MIN_WITHDRAW_AMOUNT = 0.00001 *1e18

MAX_LOCK_DURATION = 365 *60*60*24
DURATION_FACTOR = 365 *60*60*24
BOOST_WEIGHT = 100 * 1e10

UNLOCK_FREE_DURATION = 7 *60*60*24
DURATION_FACTOR_OVERDUE = 180 *60*60*24

performanceFee = 200  # 2%
performanceFeeContract = 200  # 2%
withdrawFee = 10 # 0.1%
withdrawFeeContract = 10  # 0.1%
overdueFee = 100 * 1e10   # 100%
withdrawFeePeriod = 72 *60*60  # 3 days


class BoostPool(object):

    # def __init__(self, rewardDistributor: StakingRewards) -> None:
    def __init__(
        self,
        balanceOfToken,
        totalShares,
        totalBoostDebt,
        totalLockedAmount,
        userInfo,
        blockTimestamp
    ) -> None:
        self.address = 'boostPool'
        self.balanceOfToken = balanceOfToken

        self.userInfo = {"address_0": UserInfo(), "alice": userInfo}

        # self.rewardDistributor = rewardDistributor
        self.totalShares = totalShares
        self.totalBoostDebt = totalBoostDebt
        self.totalLockedAmount = totalLockedAmount

        self.harvestRewards = 0
        self.nftBoostWeight = 0

        self.blockTimestamp = blockTimestamp
      
    def deposit(self, amount, lockDuration, to):
        user = self.userInfo[to]
        totalLockDuration = lockDuration
        if user.lockEndTime >= self.blockTimestamp:
            if amount > 0:
                user.lockStartTime = self.blockTimestamp
                self.totalLockedAmount -= user.lockedAmount
                user.lockedAmount = 0
            totalLockDuration += user.lockEndTime - user.lockStartTime
        assert lockDuration == 0 or totalLockDuration >= MIN_LOCK_DURATION, "Minimum lock period is one week"
        assert totalLockDuration <= MAX_LOCK_DURATION, "Maximum lock period exceeded"

        self._harvest()

        # if self.totalShares == 0:
        #     stockAmount = self.available()
        
        # 与reward合约关联的地方
        self._updateExtraReward(to)
        
        self._updateUserShare(to)

        if lockDuration > 0:
            if user.lockEndTime < self.blockTimestamp:
                user.lockStartTime = self.blockTimestamp
                user.lockEndTime = self.blockTimestamp + lockDuration
            else:
                user.lockEndTime += lockDuration
            user.locked = True
        
        currentShares = 0
        currentAmount = 0
        userCurrentLockedBalance = 0
        pool = self._balanceOf()
        if amount > 0:
            self.balanceOfToken += amount
            currentAmount = amount
        if user.shares > 0 and user.locked:
            userCurrentLockedBalance = (pool * user.shares) / self.totalShares
            currentAmount += userCurrentLockedBalance
            self.totalShares += user.shares
            user.shares = 0
            if user.lockStartTime == self.blockTimestamp:
                user.lockedAmount = userCurrentLockedBalance
                self.totalLockedAmount + user.lockedAmount
        if self.totalShares != 0:
            currentShares = (currentAmount * self.totalShares) / (pool - userCurrentLockedBalance)
        else:
            currentShares = currentAmount
        
        nftBoostWeight = self.nftBoostWeight

        if user.lockEndTime > user.lockStartTime:
            boostWeight = ((user.lockEndTime - user.lockStartTime) * BOOST_WEIGHT)/DURATION_FACTOR
            boostShares = (boostWeight * currentShares)/PRECISION_FACTOR
            currentShares += boostShares

            extBoostShares = (nftBoostWeight * currentShares) / PRECISION_FACTOR
            currentShares += extBoostShares
            user.shares += currentShares

            userBoostedShare = currentAmount * ((boostWeight + nftBoostWeight) * PRECISION_FACTOR + boostWeight * extBoostShares) / (PRECISION_FACTOR * PRECISION_FACTOR)
            user.userBoostedShare += userBoostedShare
            self.totalBoostDebt += userBoostedShare

            user.lockedAmount += amount
            self.totalLockedAmount += amount
            print('==> lock', to, user.lockedAmount, user.shares)
        else:
            extBoostShares = (nftBoostWeight * currentShares) / PRECISION_FACTOR
            print("not locking, extBoostShares", extBoostShares)
            currentShares += extBoostShares
            user.shares += currentShares
            userBoostedShare = (nftBoostWeight * currentAmount) / PRECISION_FACTOR
            user.userBoostedShare += userBoostedShare
            self.totalBoostDebt += userBoostedShare
        
        if amount > 0 or lockDuration > 0:
            user.lastDepositedTime = self.blockTimestamp
          
        self.totalShares += currentShares

        # Bug: user.tokenAtLastUserAction = (user.shares * self.balanceOfToken) / self.totalShares - user.userBoostedShare
        user.tokenAtLastUserAction = (user.shares * self._balanceOf()) / self.totalShares - user.userBoostedShare
        user.lastUserActionTime = self.blockTimestamp

    def withdraw(self, shares, amount, to):
        user = self.userInfo[to]
        assert shares <= user.shares, "Withdraw amount exceeds balance"
        assert user.lockEndTime < self.blockTimestamp, "Still in lock"

        currentShares = shares
        sharesPercent = shares * PRECISION_FACTOR_SHARE / user.shares

        self._harvest()

        # 与reward合约关联的地方
        self._updateExtraReward(to)

        self._updateUserShare(to)

        if shares == 0 and amount > 0:
            pool = self._balanceOf()
            currentShares = amount * self.totalShares / pool
            if currentShares > user.shares:
                currentShares = user.shares
        else:
            currentShares = sharesPercent * user.shares / PRECISION_FACTOR_SHARE
        
        outAmount = self._balanceOf() * user.shares / self.totalShares
        self.balanceOfToken -= outAmount

        user.shares -= currentShares
        self.totalShares -= currentShares

        if user.shares > 0:
            currentAmount = self._balanceOf() * user.shares / self.totalShares
            nftBoostWeight = self.nftBoostWeight
            extBoostShares = (nftBoostWeight * user.shares) / PRECISION_FACTOR
            user.shares += extBoostShares
            self.totalShares += extBoostShares
            userBoostedShare = (nftBoostWeight * currentAmount) / PRECISION_FACTOR
            user.userBoostedShare += userBoostedShare
            self.totalBoostDebt += userBoostedShare
            

            user.tokenAtLastUserAction = user.shares * self._balanceOf() / self.totalShares - user.userBoostedShare
        else:
            user.tokenAtLastUserAction = 0
        
        user.lastUserActionTime = self.blockTimestamp

        return outAmount

    def unlock(self, to):
        user = self.userInfo[to]
        assert user.locked == True, "Not locked"

        self.deposit(0, 0, to)
      
    def _harvest(self):
        # self.harvestRewards += self.rewardDistributor.getReward(self.address)
        return
    
    def _updateExtraReward(self, account):
        # self.rewardDistributor.update(account)
        return 
    
    def _updateUserShare(self, to):
        user = self.userInfo[to]
        if user.shares > 0:
            if user.userBoostedShare > 0:
                # Bug: currentAmount = (self.balanceOfToken * user.shares) / self.totalShares - user.userBoostedShare
                currentAmount = (self._balanceOf() * user.shares) / self.totalShares - user.userBoostedShare
                self.totalBoostDebt -= user.userBoostedShare
                user.userBoostedShare = 0
                self.totalShares -= user.shares
                
                if user.locked and (user.lockEndTime + UNLOCK_FREE_DURATION) < self.blockTimestamp:
                    earnAmount = currentAmount - user.lockedAmount
                    overdueDuration = self.blockTimestamp - user.lockEndTime - UNLOCK_FREE_DURATION
                    if overdueDuration > DURATION_FACTOR_OVERDUE:
                        overdueDuration = DURATION_FACTOR_OVERDUE
                    
                    overdueWeight = (overdueDuration * overdueFee) / DURATION_FACTOR_OVERDUE
                    currentOverdueFee = (earnAmount * overdueWeight) / PRECISION_FACTOR
                    # token.safeTransfer(treasury, currentOverdueFee)
                    currentAmount -= currentOverdueFee

                pool = self._balanceOf()
                currentShares = 0
                if self.totalShares != 0:
                    currentShares = (currentAmount * self.totalShares) / (pool - currentAmount)
                else:
                    currentShares = currentAmount
                user.shares = currentShares
                self.totalShares += currentShares

                if user.locked and user.lockEndTime < self.blockTimestamp:
                    user.locked = False
                    user.lockStartTime = 0
                    user.lockEndTime = 0
                    self.totalLockedAmount -= user.lockedAmount
                    user.lockedAmount = 0
                    print('==> unlock', to, currentAmount)
    
    def _balanceOf(self):
        return self.balanceOfToken + self.totalBoostDebt
        
    
    # def blockTimestamp(self):
    #     """Akin to `block.timestamp`"""
    #     return calendar.timegm(time.gmtime())
    

### Staking without Lock

### Not lock 

```
blockTimestamp 1675926955
totalShares BigNumber { value: "576477142761960289898" }
totalBoostDebt BigNumber { value: "160366031761960289898" }
totalLockedAmount BigNumber { value: "316111111000000000000" }
userInfo [
  BigNumber { value: "100000000000000000000" },
  BigNumber { value: "1675840013" },
  BigNumber { value: "100000000000000000000" },
  BigNumber { value: "1675840013" },
  BigNumber { value: "0" },
  BigNumber { value: "0" },
  BigNumber { value: "0" },
  false,
  BigNumber { value: "0" },
  shares: BigNumber { value: "100000000000000000000" },
  lastDepositedTime: BigNumber { value: "1675840013" },
  tokenAtLastUserAction: BigNumber { value: "100000000000000000000" },
  lastUserActionTime: BigNumber { value: "1675840013" },
  lockStartTime: BigNumber { value: "0" },
  lockEndTime: BigNumber { value: "0" },
  userBoostedShare: BigNumber { value: "0" },
  locked: false,
  lockedAmount: BigNumber { value: "0" }
]
balanceOf BigNumber { value: "576477142761960289898" }
==== stake ====
blockTimestamp 1675927145
totalShares BigNumber { value: "586477142761960289898" }
totalBoostDebt BigNumber { value: "160366031761960289898" }
totalLockedAmount BigNumber { value: "316111111000000000000" }
userInfo [
  BigNumber { value: "110000000000000000000" },
  BigNumber { value: "1675927145" },
  BigNumber { value: "110000000000000000000" },
  BigNumber { value: "1675927145" },
  BigNumber { value: "0" },
  BigNumber { value: "0" },
  BigNumber { value: "0" },
  false,
  BigNumber { value: "0" },
  shares: BigNumber { value: "110000000000000000000" },
  lastDepositedTime: BigNumber { value: "1675927145" },
  tokenAtLastUserAction: BigNumber { value: "110000000000000000000" },
  lastUserActionTime: BigNumber { value: "1675927145" },
  lockStartTime: BigNumber { value: "0" },
  lockEndTime: BigNumber { value: "0" },
  userBoostedShare: BigNumber { value: "0" },
  locked: false,
  lockedAmount: BigNumber { value: "0" }
]
balanceOf BigNumber { value: "586477142761960289898" }
```

In [14]:
# Init 
# userInfo [
#   shares: BigNumber { value: "100000000000000000000" },
#   lastDepositedTime: BigNumber { value: "1675840013" },
#   tokenAtLastUserAction: BigNumber { value: "100000000000000000000" },
#   lastUserActionTime: BigNumber { value: "1675840013" },
#   lockStartTime: BigNumber { value: "0" },
#   lockEndTime: BigNumber { value: "0" },
#   userBoostedShare: BigNumber { value: "0" },
#   locked: false,
#   lockedAmount: BigNumber { value: "0" }
# ]

userInfo = UserInfo(
    shares=100000000000000000000,
    lastDepositedTime=1675840013,
    tokenAtLastUserAction=100000000000000000000,
    lastUserActionTime=1675840013,
    lockStartTime=0,
    lockEndTime=0,
    userBoostedShare=0,
    locked=False,
    lockedAmount=0
)

totalShares=576477142761960289898
totalBoostDebt=160366031761960289898
totalLockedAmount=316111111000000000000
balanceOf = 576477142761960289898

balanceOfToken = balanceOf - totalBoostDebt
blockTimestamp = 1675927145

"""
balanceOfToken,
totalShares,
totalBoostDebt,
totalLockedAmount,
userInfo,
blockTimestamp
"""
pool = BoostPool(balanceOfToken,totalShares,totalBoostDebt,totalLockedAmount,userInfo,blockTimestamp)

# Alice staking
alice = "alice"
stakingAmount = 10e18

pool.deposit(stakingAmount, 0, alice)

print("user infos", pool.userInfo[alice].getAll())

print("pool balanceOfToken", format(pool.balanceOfToken, '.2f'))
print("pool totalShares", format(pool.totalShares, '.2f'))
print("pool totalBoostDebt", format(pool.totalBoostDebt, '.2f'))
print("pool totalLockedAmount", format(pool.totalLockedAmount, '.2f'))
print("pool harvestRewards", format(pool.harvestRewards, '.2f'))

not locking, extBoostShares 0.0
user infos {'shares': '110000000000000000000.00', 'lastDepositedTime': 1675927145, 'tokenAtLastUserAction': 1.0999999999999998e+20, 'lastUserActionTime': 1675927145, 'lockStartTime': 0, 'lockEndTime': 0, 'userBoostedShare': '0.00', 'locked': False, 'lockedAmount': '0.00'}
pool balanceOfToken 426111110999999971328.00
pool totalShares 586477142761960308736.00
pool totalBoostDebt 160366031761960304640.00
pool totalLockedAmount 316111110999999971328.00
pool harvestRewards 0.00


Expect:
```
blockTimestamp 1675927145
totalShares BigNumber { value: "586477142761960289898" }
totalBoostDebt BigNumber { value: "160366031761960289898" }
totalLockedAmount BigNumber { value: "316111111000000000000" }
userInfo [
  shares: BigNumber { value: "110000000000000000000" },
  lastDepositedTime: BigNumber { value: "1675927145" },
  tokenAtLastUserAction: BigNumber { value: "110000000000000000000" },
  lastUserActionTime: BigNumber { value: "1675927145" },
  lockStartTime: BigNumber { value: "0" },
  lockEndTime: BigNumber { value: "0" },
  userBoostedShare: BigNumber { value: "0" },
  locked: false,
  lockedAmount: BigNumber { value: "0" }
]
balanceOf BigNumber { value: "586477142761960289898" }
```

In [15]:
# # Alice withdraw

# shares = 1.499999999999e+20

# outAmount = pool.withdraw(shares, 0, alice)
# print("withdraw token amount", outAmount)
# print("after withdraw user infos", pool.userInfo[alice].getAll())

# print("pool balanceOfToken", pool.balanceOfToken)
# print("pool totalShares", pool.totalShares)
# print("pool totalBoostDebt", pool.totalBoostDebt)
# print("pool totalLockedAmount", pool.totalLockedAmount)
# print("pool harvestRewards", pool.harvestRewards)


### Staking with lock

```
blockTimestamp 1675927145
totalShares BigNumber { value: "586477142761960289898" }
totalBoostDebt BigNumber { value: "160366031761960289898" }
totalLockedAmount BigNumber { value: "316111111000000000000" }
userInfo [
  BigNumber { value: "110000000000000000000" },
  BigNumber { value: "1675927145" },
  BigNumber { value: "110000000000000000000" },
  BigNumber { value: "1675927145" },
  BigNumber { value: "0" },
  BigNumber { value: "0" },
  BigNumber { value: "0" },
  false,
  BigNumber { value: "0" },
  shares: BigNumber { value: "110000000000000000000" },
  lastDepositedTime: BigNumber { value: "1675927145" },
  tokenAtLastUserAction: BigNumber { value: "110000000000000000000" },
  lastUserActionTime: BigNumber { value: "1675927145" },
  lockStartTime: BigNumber { value: "0" },
  lockEndTime: BigNumber { value: "0" },
  userBoostedShare: BigNumber { value: "0" },
  locked: false,
  lockedAmount: BigNumber { value: "0" }
]
balanceOf BigNumber { value: "586477142761960289898" }
==== stake ====
blockTimestamp 1675929449
totalShares BigNumber { value: "598778512624880289898" }
totalBoostDebt BigNumber { value: "162667401624880289898" }
totalLockedAmount BigNumber { value: "436111111000000000000" }
userInfo [
  BigNumber { value: "122301369862920000000" },
  BigNumber { value: "1675929449" },
  BigNumber { value: "120000000000000000000" },
  BigNumber { value: "1675929449" },
  BigNumber { value: "1675929449" },
  BigNumber { value: "1676534249" },
  BigNumber { value: "2301369862920000000" },
  true,
  BigNumber { value: "120000000000000000000" },
  shares: BigNumber { value: "122301369862920000000" },
  lastDepositedTime: BigNumber { value: "1675929449" },
  tokenAtLastUserAction: BigNumber { value: "120000000000000000000" },
  lastUserActionTime: BigNumber { value: "1675929449" },
  lockStartTime: BigNumber { value: "1675929449" },
  lockEndTime: BigNumber { value: "1676534249" },
  userBoostedShare: BigNumber { value: "2301369862920000000" },
  locked: true,
  lockedAmount: BigNumber { value: "120000000000000000000" }
]
balanceOf BigNumber { value: "598778512624880289898" }
```

In [16]:
# Init 
# userInfo [
#   shares: BigNumber { value: "110000000000000000000" },
#   lastDepositedTime: BigNumber { value: "1675927145" },
#   tokenAtLastUserAction: BigNumber { value: "110000000000000000000" },
#   lastUserActionTime: BigNumber { value: "1675927145" },
#   lockStartTime: BigNumber { value: "0" },
#   lockEndTime: BigNumber { value: "0" },
#   userBoostedShare: BigNumber { value: "0" },
#   locked: false,
#   lockedAmount: BigNumber { value: "0" }
# ]

userInfo = UserInfo(
    shares=110000000000000000000,
    lastDepositedTime=1675927145,
    tokenAtLastUserAction=110000000000000000000,
    lastUserActionTime=1675927145,
    lockStartTime=0,
    lockEndTime=0,
    userBoostedShare=0,
    locked=False,
    lockedAmount=0
)

totalShares=586477142761960289898
totalBoostDebt=160366031761960289898
totalLockedAmount=316111111000000000000
balanceOf = 586477142761960289898

balanceOfToken = balanceOf - totalBoostDebt
blockTimestamp = 1675929449

"""
balanceOfToken,
totalShares,
totalBoostDebt,
totalLockedAmount,
userInfo,
blockTimestamp
"""
pool = BoostPool(balanceOfToken,totalShares,totalBoostDebt,totalLockedAmount,userInfo,blockTimestamp)

# Alice staking
alice = "alice"
stakingAmount = 10e18
lockDuration = 7*24*60*60
pool.deposit(stakingAmount, lockDuration, alice)

print("user infos", pool.userInfo[alice].getAll())

print("pool balanceOfToken", format(pool.balanceOfToken, '.2f'))
print("pool totalShares", format(pool.totalShares, '.2f'))
print("pool totalBoostDebt", format(pool.totalBoostDebt, '.2f'))
print("pool totalLockedAmount", format(pool.totalLockedAmount, '.2f'))
print("pool harvestRewards", format(pool.harvestRewards, '.2f'))

==> lock alice 1.2e+20 1.787706082694926e+20
user infos {'shares': '178770608269492584448.00', 'lastDepositedTime': 1675929449, 'tokenAtLastUserAction': 1.2000000000000002e+20, 'lastUserActionTime': 1675929449, 'lockStartTime': 1675929449, 'lockEndTime': 1676534249, 'userBoostedShare': '2301369863013698816.00', 'locked': True, 'lockedAmount': '120000000000000000000.00'}
pool balanceOfToken 436111110999999971328.00
pool totalShares 875247751031452860416.00
pool totalBoostDebt 162667401624974000128.00
pool totalLockedAmount 326111110999999971328.00
pool harvestRewards 0.00


Expect:
```
totalShares BigNumber { value: "598778512624880289898" }
totalBoostDebt BigNumber { value: "162667401624880289898" }
totalLockedAmount BigNumber { value: "436111111000000000000" }
userInfo [
  BigNumber { value: "120000000000000000000" },
  shares: BigNumber { value: "122301369862920000000" },
  lastDepositedTime: BigNumber { value: "1675929449" },
  tokenAtLastUserAction: BigNumber { value: "120000000000000000000" },
  lastUserActionTime: BigNumber { value: "1675929449" },
  lockStartTime: BigNumber { value: "1675929449" },
  lockEndTime: BigNumber { value: "1676534249" },
  userBoostedShare: BigNumber { value: "2301369862920000000" },
  locked: true,
  lockedAmount: BigNumber { value: "120000000000000000000" }
]
balanceOf BigNumber { value: "598778512624880289898" }
```

In [ ]:
# # after MIN_LOCK_DURATION to withdraw
# unlockMinDate = datetime.fromtimestamp(pool.userInfo[alice].getAll()['lastDepositedTime']) + timedelta(seconds=MIN_LOCK_DURATION)
# print("min unlock datetime", unlockMinDate)

# shares = 1.0000057077635572e+20

# outAmount = pool.withdraw(shares, 0, alice)
# print("unlock and withdraw token amount", outAmount)
# print("after withdraw user infos", pool.userInfo[alice].getAll())

# print("pool balanceOfToken", pool.balanceOfToken)
# print("pool totalShares", pool.totalShares)
# print("pool totalBoostDebt", pool.totalBoostDebt)
# print("pool totalLockedAmount", pool.totalLockedAmount)
# print("pool harvestRewards", pool.harvestRewards)



min unlock datetime 2023-02-02 03:26:44
==> unlock alice 9.999942916656634e+19
unlock and withdraw token amount 1e+20
after withdraw user infos {'shares': 0.0, 'lastDepositedTime': 1675308224, 'tokenAtLastUserAction': 0, 'lastUserActionTime': 1675308411, 'lockStartTime': 0, 'lockEndTime': 0, 'userBoostedShare': 0, 'locked': False, 'lockedAmount': 0}
pool balanceOfToken 0.0
pool totalShares 0.0
pool totalBoostDebt 0.0
pool totalLockedAmount 0.0
pool harvestRewards 0.0


datetime.datetime(2023, 2, 2, 3, 26, 46, 349463)